# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [9]:
# Import necessary packages
import sqlite3

In [10]:
# Create the database school.sqlite 
conn = sqlite3.connect('school.sqlite')
c = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [11]:
# Your code here
# Create the contactInfo table
c.execute('''
    CREATE TABLE contactInfo (
        userId INTEGER PRIMARY KEY AUTOINCREMENT,
        firstName TEXT NOT NULL,
        lastName TEXT NOT NULL,
        role TEXT NOT NULL,
        telephone TEXT,
        street TEXT,
        city TEXT,
        state TEXT,
        zipcode TEXT
    )
''')

OperationalError: table contactInfo already exists

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [ ]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [ ]:
# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    c.execute('''
        INSERT INTO contactInfo (firstName, lastName, role, telephone, street, city, state, zipcode)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        contact.get('firstName'),
        contact.get('lastName'),
        contact.get('role'),
        contact.get('telephone'),
        contact.get('street'),
        contact.get('city'),
        contact.get('state'),
        contact.get('zipcode')
    ))


**Query the Table to Ensure it is populated**

In [ ]:
# Your code here 
c.execute('SELECT * FROM contactInfo')
rows = c.fetchall()
for row in rows:
    print(row)

## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [ ]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [ ]:
# Create the grades table
c.execute('''
    CREATE TABLE grades (
        userId INTEGER NOT NULL,
        courseId INTEGER NOT NULL,
        grade TEXT,
        PRIMARY KEY (userId, courseId),
        FOREIGN KEY (userId) REFERENCES contactInfo(userId)
    )
''')

# Commit the changes
conn.commit()

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [ ]:
# Find the duplicate entry
c.execute('''
    SELECT firstName, lastName, COUNT(*)
    FROM contactInfo
    GROUP BY firstName, lastName
    HAVING COUNT(*) > 1
''')
duplicates = c.fetchall()
print("Duplicates:", duplicates)

In [ ]:
# Delete the duplicate entry
if duplicates:
    for duplicate in duplicates:
        firstName, lastName = duplicate[0], duplicate[1]
        c.execute('''
            DELETE FROM contactInfo
            WHERE rowid NOT IN (
                SELECT MIN(rowid)
                FROM contactInfo
                WHERE firstName = ? AND lastName = ?
            )
        ''', (firstName, lastName))


In [ ]:
# Check that the duplicate entry was removed
c.execute('SELECT * FROM contactInfo')
rows = c.fetchall()

for row in rows:
    print(row)

## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [ ]:
# Update Ed's address
c.execute('''
    UPDATE contactInfo
    SET street = '2910 Simpson Avenue', city = 'York', state = 'PA', zipcode = '17403'
    WHERE firstName = 'Ed' AND lastName = 'Lyman'
''')

In [ ]:
# Query the database to ensure the change was made
c.execute('SELECT * FROM contactInfo WHERE firstName = "Ed" AND lastName = "Lyman"')
rows = c.fetchall()

for row in rows:
    print(row)

## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [ ]:
# Your code here
conn.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 